Data Set: https://ieee-dataport.org/open-access/cad-pe
(requires log in credentials)

# Importing Libraries

In [1]:
import matplotlib.pyplot as plt
import nrrd
import numpy as np
import os
import cv2

# Reading the nrrd files and Creating slices

## Function to save each slice from an nrrd file

In [124]:
# Applying the default values of medical masking to concentrate in the central aorta depiction
def save_slices(nrrd_file_path, output_folder, lower_bound, upper_bound):
    # Reading the nrrd file
    # nrrd.read(nrrd_file_path) returns data, header
    # header includes a dictionary with information about the nrrd file (type, sizes, encoding)
    data, _ = nrrd.read(nrrd_file_path)
    data_rotated = np.rot90(data, axes=(1, 0))  # Apply rotation

    # If output folder doesn't exists
    if not os.path.exists(output_folder):
        # It is created
        os.makedirs(output_folder)

    # Processing and saving each slice
    for slice_number in range(data_rotated.shape[2]):
        # data_rotated.shape[0] : X dimension
        # data_rotated.shape[1] : Y dimension
        # data_rotated.shape[2] : Number of slices
        # Selecting each slice from the nrrd file
        slice = data_rotated[:, :, slice_number]

        # Applying lower bound threshold
        _, lower_thresh = cv2.threshold(slice, lower_bound, 255, cv2.THRESH_TOZERO)

        # Applying upper bound threshold (inverting the image first)
        inverted_slice = 255 - lower_thresh
        _, upper_thresh_inv = cv2.threshold(inverted_slice, 255 - upper_bound, 255, cv2.THRESH_TOZERO)
        processed_slice = 255 - upper_thresh_inv

        # Saving the processed slice
        plt.imsave(os.path.join(output_folder, f"slice_{slice_number}.png"), processed_slice, cmap='gray')

## Creating slices for patients

### Individually

In [ ]:
patient_number = '022'
nrrd_file_path = f'patients_nrrd/{patient_number}.nrrd'
output_folder = f'patient_{patient_number}'
save_slices(nrrd_file_path, output_folder, lower_bound=142-730/2, upper_bound=142+730/2)

### Iteratively

In [ ]:
# for i in range(1, 834): # 1st patient number is 1 and last is 833
#     if i < 10:
#         patient_number = f'00{i}'
#         nrrd_file_path = f'../{patient_number}.nrrd'
#         output_folder = f'patient_{patient_number}'
#         save_slices(nrrd_file_path, output_folder, lower_bound=142-730/2, upper_bound=142+730/2)
#         continue
#     if i < 41: # patient numbers stop at 40 and continue after 100
#         patient_number = f'0{i}'
#         nrrd_file_path = f'../{patient_number}.nrrd'
#         output_folder = f'patient_{patient_number}'
#         save_slices(nrrd_file_path, output_folder, lower_bound=142-730/2, upper_bound=142+730/2)
#         continue
#     if i > 100:
#         try: # over 100 numbers not consistent
#             patient_number = f'{i}'
#             nrrd_file_path = f'../e0{patient_number}.nrrd' # After 100, nrrd files are named e0###.nrrd
#             output_folder = f'patient_{patient_number}'
#             save_slices(nrrd_file_path, output_folder, lower_bound=142-730/2, upper_bound=142+730/2)
#         except:
#             continue

-------------------------------------------------------------

# Listing the slices containing central aorta 
## for a single patient by identifying the ascending thoracic aorta

In [2]:
slices_list = []

patient_number = '006'

try:
    slices = os.listdir(f'patient_{patient_number}')
except:
    print(f'No slices folder for patient {patient_number}')


if len(slices) == 0:
    print(f"Patient {patient_number}'s slices folder is empty")
else: # slices contains ['slice_0.png', 'slice_1.png', ... ,'slice_N.png']
    for i in range(len(slices)): # for i range(N+1)
        slice = cv2.imread(f'patient_{patient_number}/slice_{i}.png',0)

        ## Apply Gaussian blur
        blurred_slice = cv2.GaussianBlur(slice, (31, 31), 2)
        
        ## Cropping the blurred slice 
        ## to look for the ascending thoracic aorta more efficiently

        # Defining where the cropped slice should start and end
        # because the ascending thoracic aorta appears in the center of the slice
        start = 0.25
        end = 0.75        
        
        # Cropping
        cropped = blurred_slice.copy()[int(blurred_slice.shape[1]*(start)):int(blurred_slice.shape[1]*(end-0.15)),
                                       int(blurred_slice.shape[0]*(start)):int(blurred_slice.shape[0]*end)]

        ## Detecting circles using HoughCircles
        ## to look for the ascending thoracic aorta
        circles = cv2.HoughCircles(cropped, cv2.HOUGH_GRADIENT, dp=1, minDist=100,
                                param1=30, param2=35, minRadius=20, maxRadius=35)
        
        # If circles are detected,
        # the slice contains the ascending thoracic aorta,
        # append the slice number in the list of slices
        if circles is not None:
            slices_list.append(i)


# Processing the slices of the central aorta to locate the embolism
## Using the above list

In [3]:
# Setting an output folder
# for the processed slices
# based on patient number from above
output_folder = f'patient_{patient_number}_processed'
if not os.path.exists(output_folder):
        os.makedirs(output_folder)


# Using the center of the list
# keeping only 31 slices from the slices list
list_start = slices_list[int(len(slices_list)/2)] - 15
list_end = slices_list[int(len(slices_list)/2)] + 16    

for i in range(list_start,list_end):
    slice = cv2.imread(f'patient_{patient_number}/slice_{i}.png',0)

    # Creating copies for later use
    slice_copy_1 = slice.copy()
    slice_copy_2 = slice.copy()
    
    ##############################################
    ### Mapping the central aorta in the slice ###

    ## 'Cropping' the slice 
    ## to look for the central aorta more efficiently

    # Defining where the 'cropped' slice should start and end
    start = 0.25
    end = 0.75

    # Defining the corners of the 'cropped' slice
    upper_left = [int(slice.shape[0]*(start)), int(slice.shape[1]*(start))]
    upper_right = [int(slice.shape[0]*end), int(slice.shape[1]*(start))]
    lower_left = [int(slice.shape[0]*(start)), int(slice.shape[1]*(end-0.15))]
    lower_right = [int(slice.shape[0]*end),int(slice.shape[1]*(end-0.15))]

    for x in range(slice.shape[0]):
        for y in range(slice.shape[1]):
            if (y not in range(upper_left[0],upper_right[0])) or (x not in range(upper_left[1],lower_left[1])):
                slice[x,y] = 0

    ## Applying binary threshold to the cropped slice
    ## to remove the background

    _, slice_binary =cv2.threshold(slice,115,255,cv2.THRESH_BINARY)


    ## Applying connected components analysis
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(slice_binary)

    # The background is the 1st and largest component
    # hence from stats the 1st compenent is excluded
    # The np.argmax() looks for the index of the largest component excluding the background
    # Therefore the largest component index from stats would be the above +1
    largest_component_index = np.argmax(stats[1:, cv2.CC_STAT_AREA])+1

    # Creating a binary mask of the largest component
    largest_component_mask = (labels == largest_component_index).astype(np.uint8) * 255

    ## Eroding the largest component

    # The largest component acquired from above
    # could contain the ascending thoracic aorta (not needed)
    # along with the central aorta (need)
    # Eroding the component 
    # and applying another connected components analysis
    # creates a mask of the central aorta

    kernel = np.ones((3, 3), np.uint8)

    eroded_component = cv2.erode(largest_component_mask, kernel, iterations=2)

    ## Connected components analysis
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(eroded_component)
    largest_component_index = np.argmax(stats[1:, cv2.CC_STAT_AREA])+1
    largest_component_mask = (labels == largest_component_index).astype(np.uint8) * 255

    ### Central aorta acquired ###
    #################################################
    ### Mapping the embolism on the central aorta ###


    ## 'Cropping' the component
    ## to keep the bottom half of the central aorta

    # There is a need to dilate the component to include the embolism
    # that exist on the bottom half of the central aorta.
    # By cropping the component first
    # the dilation has an effect on the bottom half only.

    # 'Cropping' the component up from x = 0 up to x = middle of component
    for x in range(0,stats[largest_component_index][1] + int(stats[largest_component_index][3]/2)):
        largest_component_mask[x,:] = 0

    ## Dilating to include the embolism
    dilated_component = cv2.dilate(largest_component_mask, kernel, iterations=6)


    ## Masking the central aorta and its embolism
    slice_copy_1[dilated_component == 0] = 0

    ## Bluring the central aorta and its embolism
    blurred_aorta = cv2.GaussianBlur(slice_copy_1,(5,5),0)

    ## to use a more effective band thresholding on the embolism
    ## and create a mask containg the embolisms
    embolism_mask = cv2.inRange(blurred_aorta, 90, 120)

    ## Eroding to remove small particles around the embolism
    ## that went through the band thresholding
    eroded_embolism_component = cv2.erode(embolism_mask, kernel, iterations=1)

    ## Connected components analysis
    ## to keep the largest component
    ## that is the embolism
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(eroded_embolism_component)
    largest_component_indices = np.argsort(stats[1:, cv2.CC_STAT_AREA]) + 1
    embolism_mask = (labels == largest_component_index).astype(np.uint8) * 255

    ## Dilating the embolisk mask
    ## to reverse the previous erosion
    embolism_mask = cv2.dilate(embolism_mask, kernel, iterations=2)

    ### Embolism Acquired ###
    #############################
    ### Coloring the embolism ###

    ## On a RGB version of the slice
    slice_rgb = cv2.cvtColor(slice_copy_2, cv2.COLOR_GRAY2RGB)

    ## Using red color
    color = (0, 0, 255)  # BGR format (red)

    # Applying the color to the corresponding pixel in the original image
    slice_rgb[embolism_mask != 0] = color

    ##### Saving the colored/processed slice #####
    cv2.imwrite(f'{output_folder}/slice_{i}_processed.png', slice_rgb)